Final cleaned version

In [3]:
import pandas as pd
import panel as pn
import plotly.express as px
import plotly.io as pio

# Set default Plotly theme
pio.templates.default = "plotly_dark"

# Load and preprocess the data
file_path = r'D:\Project_Tuna_Fisheries_Data_Entry\Data\Python\TFDF File Main database_09-12-2024.xlsx'

try:
    data = pd.read_excel(file_path, sheet_name='Sheet1')
except (FileNotFoundError, ValueError):
    print("Error loading data. Falling back to default sample dataset.")
    data = pd.DataFrame({
        'Reg No.': ['A1', 'B2', 'C3'],
        'Year': [2018, 2019, 2020],
        'Total sheets': [10, 20, 30]
    })

# Clean and preprocess the data
if not {'Reg No.', 'Year', 'Total sheets'}.issubset(data.columns):
    raise KeyError("Required columns are missing from the data. Ensure 'Reg No.', 'Year', and 'Total sheets' exist.")

focused_data = data[['Reg No.', 'Year', 'Total sheets']].dropna(subset=['Reg No.', 'Year', 'Total sheets'])
if focused_data.empty:
    raise ValueError("Processed dataset is empty after cleaning. Check the data.")

# Initialize Panel extensions
pn.extension('plotly')

# Define Widgets
reg_selector = pn.widgets.MultiChoice(
    name='Registration Numbers',
    options=list(focused_data['Reg No.'].unique()),
    placeholder="Select Registration Numbers (Optional)"
)
year_selector = pn.widgets.IntRangeSlider(
    name='Year Range',
    start=int(focused_data['Year'].min()),
    end=int(focused_data['Year'].max()),
    value=(int(focused_data['Year'].min()), int(focused_data['Year'].max()))
)
total_sheets_slider = pn.widgets.IntRangeSlider(
    name='Total Sheets Range',
    start=int(focused_data['Total sheets'].min()),
    end=int(focused_data['Total sheets'].max()),
    value=(int(focused_data['Total sheets'].min()), int(focused_data['Total sheets'].max()))
)
theme_toggle = pn.widgets.Toggle(name="Dark Mode", value=True)
reset_button = pn.widgets.Button(name="Reset Filters", button_type="primary")

# Unified Filter Function
_filtered_cache = None  # Cache for filtered data

def invalidate_cache(*events):
    global _filtered_cache
    _filtered_cache = None

def filter_data():
    global _filtered_cache
    if _filtered_cache is not None:
        return _filtered_cache
    filtered = focused_data.copy()
    if reg_selector.value:
        filtered = filtered[filtered['Reg No.'].isin(reg_selector.value)]
    if year_selector.value:
        filtered = filtered[
            (filtered['Year'] >= year_selector.value[0]) & (filtered['Year'] <= year_selector.value[1])
        ]
    if total_sheets_slider.value:
        filtered = filtered[
            (filtered['Total sheets'] >= total_sheets_slider.value[0]) & 
            (filtered['Total sheets'] <= total_sheets_slider.value[1])
        ]
    _filtered_cache = filtered
    return filtered

# Reset filters to default
def reset_filters(event):
    reg_selector.value = []
    year_selector.value = (int(focused_data['Year'].min()), int(focused_data['Year'].max()))
    total_sheets_slider.value = (int(focused_data['Total sheets'].min()), int(focused_data['Total sheets'].max()))
    invalidate_cache()

reset_button.on_click(reset_filters)
reg_selector.param.watch(invalidate_cache, 'value')
year_selector.param.watch(invalidate_cache, 'value')
total_sheets_slider.param.watch(invalidate_cache, 'value')

# Update theme based on toggle
@pn.depends(theme_toggle.param.value, watch=True)
def update_theme(is_dark):
    pio.templates.default = "plotly_dark" if is_dark else "plotly_white"

# KPI Cards
def kpi_card(name, value, description):
    return pn.pane.HTML(
        f"""
        <div style='text-align: center; padding: 10px; border: 1px solid #ccc; border-radius: 10px; background-color: #f9f9f9;'>
            <h3 style='margin: 5px;'>{name}</h3>
            <p style='font-size: 24px; margin: 5px; font-weight: bold;'>{value}</p>
            <p style='font-size: 12px; margin: 5px; color: #555;'>{description}</p>
        </div>
        """,
        width=200
    )

# Display KPIs
@pn.depends(reg_selector.param.value, year_selector.param.value, total_sheets_slider.param.value)
def display_kpis(reg_numbers, year_range, sheets_range):
    filtered = filter_data()

    if filtered.empty:
        return pn.pane.Markdown("### No data matches the selected filters. Please adjust the filters.", style={"color": "red"})

    total_registrations = filtered['Reg No.'].nunique()
    total_sheets = filtered['Total sheets'].sum()
    avg_sheets = filtered['Total sheets'].mean()
    median_sheets = filtered['Total sheets'].median()
    max_sheets = filtered['Total sheets'].max()
    std_dev_sheets = filtered['Total sheets'].std()

    return pn.Row(
        kpi_card("Total Registrations", total_registrations, "Unique registrations in the filtered data"),
        kpi_card("Total Sheets", total_sheets, "Sum of all sheets in the filtered data"),
        kpi_card("Avg Sheets/Reg", f"{avg_sheets:.2f}", "Average sheets per registration"),
        kpi_card("Median Sheets/Reg", f"{median_sheets:.2f}", "Median of sheets per registration"),
        kpi_card("Max Sheets/Reg", max_sheets, "Maximum sheets by a single registration"),
        kpi_card("Std Dev Sheets", f"{std_dev_sheets:.2f}", "Standard deviation of sheets")
    )

# Create Charts
@pn.depends(reg_selector.param.value, year_selector.param.value, total_sheets_slider.param.value)
def create_charts(reg_numbers, year_range, sheets_range):
    filtered = filter_data()

    if filtered.empty:
        return pn.pane.Markdown("### No data available for the selected filters.", style={"color": "red"})

    charts = []
    charts.append(pn.pane.Plotly(px.scatter(
        filtered, x='Year', y='Total sheets', color='Reg No.',
        size='Total sheets', title='Scatter Plot: Total Sheets vs Year',
        trendline='ols', height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.histogram(
        filtered, x='Total sheets', nbins=20,
        title='Histogram: Distribution of Total Sheets',
        height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.bar(
        filtered, x='Reg No.', y='Total sheets', color='Year',
        title='Bar Chart: Total Sheets by Registration Number',
        height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.line(
        filtered.groupby('Year')['Total sheets'].sum().reset_index(),
        x='Year', y='Total sheets',
        title='Line Chart: Total Sheets Over Time',
        height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.box(
        filtered, x='Year', y='Total sheets', color='Reg No.',
        title='Box Plot: Total Sheets Distribution by Year',
        height=500, width=800
    )))
    heatmap_data = filtered.pivot_table(
        index='Reg No.', columns='Year', values='Total sheets', aggfunc='sum', fill_value=0
    )
    charts.append(pn.pane.Plotly(px.imshow(
        heatmap_data, title='Heatmap: Total Sheets by Year and Registration Number',
        color_continuous_scale='Viridis', height=500, width=800
    )))
    cumulative_data = filtered.sort_values(by='Year')
    cumulative_data['Cumulative Sheets'] = cumulative_data['Total sheets'].cumsum()
    charts.append(pn.pane.Plotly(px.line(
        cumulative_data, x='Year', y='Cumulative Sheets',
        title='Cumulative Sheets Over Time',
        height=500, width=800
    )))
    return pn.Column(*charts)

# Dashboard Layout
instructions = pn.pane.Markdown(
    "### Instructions\n1. Use filters to narrow down data.\n2. Explore visualizations in real time.\n3. Reset filters for default view.\n\n### Troubleshooting\n- If no data appears, try resetting the filters.\n- Ensure the dataset is loaded correctly and contains the required columns."
)

dashboard = pn.Column(
    pn.pane.Markdown("# Interactive Dashboard: Total Sheets Analysis"),
    instructions,
    pn.Row(
        pn.Column(reg_selector),
        pn.Column(year_selector),
        pn.Column(total_sheets_slider),
        pn.Column(theme_toggle),
        pn.Column(reset_button),
    ),
    display_kpis,
    create_charts
)

# Serve the Dashboard
dashboard.servable()


DEBUG:markdown_it.rules_block.code:entering code: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.fence:entering fence: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.blockquote:entering blockquote: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.hr:entering hr: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.list:entering list: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.reference:entering reference: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.html_block:entering html_block: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.heading:entering heading: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.code:entering code: StateBlock(line=0,level=0,tokens=0), 0, 8, False
DEBUG:markdown_it.rules_block.fence:entering fence: StateBlock(line=0,l

BokehModel(combine_events=True, render_bundle={'docs_json': {'b243ae9a-22dc-440f-af9b-737b136219aa': {'version…

DEBUG:Comm:handle_msg[d76b3cea50854e3ba95829b221558ad2]({'header': {'date': datetime.datetime(2024, 12, 13, 10, 8, 50, 20000, tzinfo=tzutc()), 'msg_id': '3f19a86e-aa3a-4d3a-bc43-0ccc052739bb', 'msg_type': 'comm_msg', 'session': '5fa71421-42a4-41f8-98d4-424506e73710', 'username': '6f4d10e5-9cd5-4a46-a24a-bd39b051fa64', 'version': '5.2'}, 'msg_id': '3f19a86e-aa3a-4d3a-bc43-0ccc052739bb', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': 'd76b3cea50854e3ba95829b221558ad2', 'data': {'method': 'update', 'state': {'_view_count': 1}, 'buffer_paths': []}}, 'buffers': []})
DEBUG:Comm:handle_msg[d76b3cea50854e3ba95829b221558ad2]({'header': {'date': datetime.datetime(2024, 12, 13, 10, 8, 50, 848000, tzinfo=tzutc()), 'msg_id': 'a7485574-6741-4416-a39b-22355cd3a30c', 'msg_type': 'comm_msg', 'session': '5fa71421-42a4-41f8-98d4-424506e73710', 'username': '6f4d10e5-9cd5-4a46-a24a-bd39b051fa64', 'version': '5.2'}, 'msg_id': 'a7485574-6741-4416-a39b-22355cd3a30c', 'msg_

with Debug

In [4]:
import pandas as pd
import panel as pn
import plotly.express as px
import plotly.io as pio
import logging

# Initialize logging
logging.basicConfig(level=logging.DEBUG)

# Set default Plotly theme
pio.templates.default = "plotly_dark"

# Load and preprocess the data
file_path = r'D:\Project_Tuna_Fisheries_Data_Entry\Data\Python\TFDF File Main database_09-12-2024.xlsx'

try:
    data = pd.read_excel(file_path, sheet_name='Sheet1')
except (FileNotFoundError, ValueError):
    print("Error loading data. Falling back to sample dataset for demonstration purposes.")
    data = pd.DataFrame({
        'Reg No.': ['A1', 'B2', 'C3'],
        'Year': [2018, 2019, 2020],
        'Total sheets': [10, 20, 30]
    })

# Validate columns
required_columns = {'Reg No.', 'Year', 'Total sheets'}
if not required_columns.issubset(data.columns):
    raise KeyError(f"Required columns are missing: {required_columns - set(data.columns)}.")

focused_data = data[['Reg No.', 'Year', 'Total sheets']].dropna(subset=required_columns)
if focused_data.empty:
    raise ValueError("Processed dataset is empty after cleaning. Check the data.")

# Initialize Panel extensions
pn.extension('plotly')

# Define Widgets
reg_selector = pn.widgets.MultiChoice(
    name='Registration Numbers',
    options=list(focused_data['Reg No.'].unique()),
    placeholder="Select Registration Numbers (Optional)"
)
year_selector = pn.widgets.IntRangeSlider(
    name='Year Range',
    start=int(focused_data['Year'].min()),
    end=int(focused_data['Year'].max()),
    value=(int(focused_data['Year'].min()), int(focused_data['Year'].max()))
)
total_sheets_slider = pn.widgets.IntRangeSlider(
    name='Total Sheets Range',
    start=int(focused_data['Total sheets'].min()),
    end=int(focused_data['Total sheets'].max()),
    value=(int(focused_data['Total sheets'].min()), int(focused_data['Total sheets'].max()))
)
theme_toggle = pn.widgets.Toggle(name="Dark Mode", value=True)
reset_button = pn.widgets.Button(name="Reset Filters", button_type="primary")

# Cache for filtered data
_filtered_cache = None

def invalidate_cache(*events):
    global _filtered_cache
    _filtered_cache = None

def filter_data():
    global _filtered_cache
    if _filtered_cache is not None:
        return _filtered_cache
    filtered = focused_data.copy()
    if reg_selector.value:
        filtered = filtered[filtered['Reg No.'].isin(reg_selector.value)]
    if year_selector.value:
        filtered = filtered[
            (filtered['Year'] >= year_selector.value[0]) & (filtered['Year'] <= year_selector.value[1])
        ]
    if total_sheets_slider.value:
        filtered = filtered[
            (filtered['Total sheets'] >= total_sheets_slider.value[0]) &
            (filtered['Total sheets'] <= total_sheets_slider.value[1])
        ]
    _filtered_cache = filtered
    return filtered

# Reset filters
def reset_filters(event):
    reg_selector.value = []
    year_selector.value = (focused_data['Year'].min(), focused_data['Year'].max())
    total_sheets_slider.value = (focused_data['Total sheets'].min(), focused_data['Total sheets'].max())
    invalidate_cache()

reset_button.on_click(reset_filters)
reg_selector.param.watch(invalidate_cache, 'value')
year_selector.param.watch(invalidate_cache, 'value')
total_sheets_slider.param.watch(invalidate_cache, 'value')

# Update theme
@pn.depends(theme_toggle.param.value, watch=True)
def update_theme(is_dark):
    pio.templates.default = "plotly_dark" if is_dark else "plotly_white"

# KPI Cards
def kpi_card(name, value, description):
    return pn.pane.HTML(
        f"""
        <div style='text-align: center; padding: 10px; border: 1px solid #ccc; border-radius: 10px; background-color: #f9f9f9;'>
            <h3 style='margin: 5px;'>{name}</h3>
            <p style='font-size: 24px; margin: 5px; font-weight: bold;'>{value}</p>
            <p style='font-size: 12px; margin: 5px; color: #555;'>{description}</p>
        </div>
        """,
        width=200
    )

# Display KPIs
@pn.depends(reg_selector.param.value, year_selector.param.value, total_sheets_slider.param.value)
def display_kpis(reg_numbers, year_range, sheets_range):
    filtered = filter_data()

    if filtered.empty:
        return pn.pane.Markdown("### No data matches the selected filters. Please adjust the filters.", style={"color": "red"})

    total_registrations = filtered['Reg No.'].nunique()
    total_sheets = filtered['Total sheets'].sum()
    avg_sheets = filtered['Total sheets'].mean()
    median_sheets = filtered['Total sheets'].median()
    max_sheets = filtered['Total sheets'].max()
    std_dev_sheets = filtered['Total sheets'].std()

    return pn.Row(
        kpi_card("Total Registrations", total_registrations, "Unique registrations in the filtered data"),
        kpi_card("Total Sheets", total_sheets, "Sum of all sheets in the filtered data"),
        kpi_card("Avg Sheets/Reg", f"{avg_sheets:.2f}", "Average sheets per registration"),
        kpi_card("Median Sheets/Reg", f"{median_sheets:.2f}", "Median of sheets per registration"),
        kpi_card("Max Sheets/Reg", max_sheets, "Maximum sheets by a single registration"),
        kpi_card("Std Dev Sheets", f"{std_dev_sheets:.2f}", "Standard deviation of sheets")
    )

# Create Charts
@pn.depends(reg_selector.param.value, year_selector.param.value, total_sheets_slider.param.value)
def create_charts(reg_numbers, year_range, sheets_range):
    filtered = filter_data()
    if filtered.empty:
        return pn.pane.Markdown(
            f"### No data available for the selected filters:\n"
            f"- Registration Numbers: {', '.join(reg_selector.value) or 'All'}\n"
            f"- Year Range: {year_selector.value}\n"
            f"- Total Sheets Range: {total_sheets_slider.value}",
            style={"color": "red"}
        )

    # Regenerate charts dynamically
    charts = []
    charts.append(pn.pane.Plotly(px.scatter(
        filtered, x='Year', y='Total sheets', color='Reg No.',
        size='Total sheets', title='Scatter Plot: Total Sheets vs Year',
        trendline='ols', height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.histogram(
        filtered, x='Total sheets', nbins=20,
        title='Histogram: Distribution of Total Sheets',
        height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.bar(
        filtered, x='Reg No.', y='Total sheets', color='Year',
        title='Bar Chart: Total Sheets by Registration Number',
        height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.line(
        filtered.groupby('Year')['Total sheets'].sum().reset_index(),
        x='Year', y='Total sheets',
        title='Line Chart: Total Sheets Over Time',
        height=500, width=800
    )))
    charts.append(pn.pane.Plotly(px.box(
        filtered, x='Year', y='Total sheets', color='Reg No.',
        title='Box Plot: Total Sheets Distribution by Year',
        height=500, width=800
    )))
    heatmap_data = filtered.pivot_table(
        index='Reg No.', columns='Year', values='Total sheets', aggfunc='sum', fill_value=0
    )
    charts.append(pn.pane.Plotly(px.imshow(
        heatmap_data, title='Heatmap: Total Sheets by Year and Registration Number',
        color_continuous_scale='Viridis', height=500, width=800
    )))
    cumulative_data = filtered.sort_values(by='Year')
    cumulative_data['Cumulative Sheets'] = cumulative_data['Total sheets'].cumsum()
    charts.append(pn.pane.Plotly(px.line(
        cumulative_data, x='Year', y='Cumulative Sheets',
        title='Cumulative Sheets Over Time',
        height=500, width=800
    )))
    return pn.Column(*charts)

# Dashboard Layout
instructions = pn.pane.Markdown(
    "### Instructions\n1. Use filters to narrow down data.\n2. Explore visualizations in real time.\n3. Reset filters for default view.\n\n### Troubleshooting\n- If no data appears, try resetting the filters.\n- Ensure the dataset is loaded correctly and contains the required columns."
)

dashboard = pn.Column(
    pn.pane.Markdown("# Interactive Dashboard: Total Sheets Analysis"),
    instructions,
    pn.Row(
        pn.Column(reg_selector),
        pn.Column(year_selector),
        pn.Column(total_sheets_slider),
        pn.Column(theme_toggle),
        pn.Column(reset_button),
    ),
    display_kpis,
    create_charts
)

# Serve the Dashboard
dashboard.servable()


DEBUG:markdown_it.rules_block.code:entering code: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.fence:entering fence: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.blockquote:entering blockquote: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.hr:entering hr: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.list:entering list: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.reference:entering reference: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.html_block:entering html_block: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.heading:entering heading: StateBlock(line=0,level=0,tokens=0), 0, 1, False
DEBUG:markdown_it.rules_block.code:entering code: StateBlock(line=0,level=0,tokens=0), 0, 8, False
DEBUG:markdown_it.rules_block.fence:entering fence: StateBlock(line=0,l

BokehModel(combine_events=True, render_bundle={'docs_json': {'23b982d1-d11d-4292-92c4-7ada7b174ee9': {'version…

DEBUG:Comm:handle_msg[d6bc21e7277c41b8b73013ebe6df32b5]({'header': {'date': datetime.datetime(2024, 12, 13, 10, 8, 29, 437000, tzinfo=tzutc()), 'msg_id': '1e80312b-5e61-4a8f-8b3c-049f7f207419', 'msg_type': 'comm_msg', 'session': '5fa71421-42a4-41f8-98d4-424506e73710', 'username': '6f4d10e5-9cd5-4a46-a24a-bd39b051fa64', 'version': '5.2'}, 'msg_id': '1e80312b-5e61-4a8f-8b3c-049f7f207419', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': 'd6bc21e7277c41b8b73013ebe6df32b5', 'data': {'method': 'update', 'state': {'_view_count': 1}, 'buffer_paths': []}}, 'buffers': []})
DEBUG:Comm:handle_msg[d6bc21e7277c41b8b73013ebe6df32b5]({'header': {'date': datetime.datetime(2024, 12, 13, 10, 8, 30, 295000, tzinfo=tzutc()), 'msg_id': '59c37ee9-8ab4-4528-b413-5150f068575b', 'msg_type': 'comm_msg', 'session': '5fa71421-42a4-41f8-98d4-424506e73710', 'username': '6f4d10e5-9cd5-4a46-a24a-bd39b051fa64', 'version': '5.2'}, 'msg_id': '59c37ee9-8ab4-4528-b413-5150f068575b', 'msg